In [21]:
# Download metadata to ./data/weather
import os
import urllib


# URLs for metadata
metadata_urls = {
    'station_inventory': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt', 
    'states': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-states.txt', 
    'stations': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt', 
    'countries': 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-countries.txt'
}

# Output dir
directory = './data/weather'
if not os.path.exists(directory):
    os.makedirs(directory)

uo = urllib.URLopener()

for k, u in metadata_urls.items():
    uo.retrieve(u, './data/weather/'+k+'.txt')

In [51]:
# Download daily weather data summary by year
import requests
import gzip
import io
import csv

# URL for yearly data 
yr_base_url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/'
yrs = range(1998, 2015+1)

for yr in yrs:
    yr_url = ''.join([yr_base_url, str(yr), '.csv.gz'])
    
    print 'downloading file for: %d...' % (yr)
    resp = requests.get(yr_url, stream=True)
    csv_gz = resp.content
    f = io.BytesIO(csv_gz)
    print 'download complete'
    
    with gzip.GzipFile(fileobj=f, mode='rb') as fh, gzip.open('./data/weather/'+str(yr)+'.csv.gz', 'wb+') as fout:
        print 'filtering file...'
        for line in fh:
            if 'US' == line[:2]:
                fout.write(line)
        print 'finished writing file for: %d...\n' % (yr)

downloading file number: 0...
download complete
filtering file number: 0...
finished writing file number: 0...



In [125]:
# Helper class for claculating distance between two lat, lon
# https://www.movable-type.co.uk/scripts/latlong.html

from math import *

class Haversine:
    def __init__(self, radius=3959.87433):
        self.RADIUS = radius
        
    def distance(self, src, dest):
        ''' Returns distance in miles
        '''
        lat1, lon1 = map(radians, src)
        lat2, lon2 = map(radians, dest)
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        
        return self.RADIUS * c
    
    def bearing(self, src, dest):
        '''Returns compass degrees
        '''
        lat1, lon1 = map(radians, src)
        lat2, lon2 = map(radians, dst)
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        y = sin(dlon) * cos(lon2)
        x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
        bearing = atan2(y, x)
        bearing = degrees(bearing)
        return (bearing + 360) % 360
    
    
if __name__ == '__main__':
    dst = (-94.5930, -39.1230)
    src = (-94.4839, -39.1561)
    h = Haversine()
    print h.distance(src, dst)
    print h.bearing(src, dst)

7.54238357394
166.75532027


In [126]:
dst = (34.0, -118.0)
src = (34.0, -73.0)
h = Haversine()
print h.distance(src, dst) * 1.6
print h.bearing(src, dst)

4090.86961415
67.7541634922
